### This file contains base functions for running the TRAK simulations in python

In [2]:
# import needed libraries
import os 
import numpy as np 
import matplotlib.pyplot as plt 
from os.path import basename
from re import sub

In [ ]:
## Function to get the current convergence plot and final current for a TRAK run (.TLS file)
def current_conv(TLS_file):  #put in path to file
    try:
        file = open(r'C://fieldp_pro/tricomp/Automation/sim_automation/%s.TLS' %TLS_file) #test this out
        lines = file.readlines()

        Current_convergence = []
        i = 0
        j = 0
        for line in lines: 
            if 'Current convergence history' in line: 
                j = 1
            if 'Ending time' in line: 
                j = 0
            if j == 1: 
                Current_convergence.append(lines[i])
            i += 1

        Current_convergence = Current_convergence[4:-1]

        NCycle = []
        Total_emitted_current = []
        for line in Current_convergence: 
            separate  = line.split()
            NCycle.append(float(separate[0]))
            Total_emitted_current.append(float(separate[1]))

        plt.plot(NCycle, Total_emitted_current, label = 'Final Current: %.3f A'%Total_emitted_current[-1])
        plt.xlabel("Iteration")
        plt.ylabel("Total Emitted Current [A]")
        plt.title('Current Convergence')
        plt.legend(handlelength=0, handletextpad=0)
        #Save the plot image!! 
        plt.savefig('%s_conv.png' %TLS_file)

        print("File: ", TLS_file ,'    ', "Final Current:", Total_emitted_current[-1])
        return(Total_emitted_current[-1])
    except: 
        return("ERROR - couldn't find file!")

In [ ]:
# Example of running current_conv function: 
#current_conv('TRAK_auto_10_03/TRAK_demit_007')

In [ ]:
## Function to plot the electric field at slices... 
## Picking the slices that will have the highest field 
def E_field_line(TLS_file, e_slice, plots = True):
    try: 
        file = open(r'C://fieldp_pro/tricomp/Automation/sim_automation/%s.TLS' %TLS_file) #test this out
        lines = file.readlines()
        
        e_slice_str = str(e_slice)
        
        E_field_data = []
        i,j = 0,0
        for line in lines: 
            if (j ==1) and ('Line' in line): 
               j = 0
            if 'ESCAN (%s'%e_slice_str in line or 'ESCAN(%s'%e_slice_str in line:   #the number to match where the cdens is
                j = 1
            if j == 1: 
                E_field_data.append(line)
            i += 1
        
        x = []
        y = []
        z = []
        Ex = []
        Ey = []
        Ez = []

        i = 0
        for line in E_field_data:
            if i > 5:
                line_new = line.replace('  ', ' ')
                line_new = line_new.replace('   ', ' ')
                line_new = line_new.split()
                x.append(float(line_new[0]))
                y.append(float(line_new[1]))
                z.append(float(line_new[2]))
                Ex.append(float(line_new[3]))
                Ey.append(float(line_new[4]))
                Ez.append(float(line_new[5]))
            i += 1
                
        E_tot = []    
        for Exval, Eyval, Ezval in zip(Ex, Ey, Ez): 
            E_tot_val = np.sqrt(Exval**2 + Eyval**2 + Ezval**2)
            E_tot.append(E_tot_val*10**(-3))
            
        ## Do this part only if plots = true    
        if plots == True: 
            #plt.show()
            plt.plot(y, Ez)
            plt.ylabel('Ez field [V/m]')
            plt.xlabel('y position')
            plt.show()

            plt.plot(y, Ex)
            plt.ylabel("Ex field [V/m]")
            plt.xlabel('y position')
            plt.show()

            plt.plot(y, Ey)
            plt.ylabel("Ey field [V/m]")
            plt.xlabel('y position')
            plt.show()

            plt.plot(y, E_tot)
            plt.ylabel("E field [kV/m]")
            plt.xlabel('R [mm]')
            plt.show()

        max_E_field = np.round(max(E_tot), 3)
        E_field_list = []
        for i in E_tot: 
            val = i
            E_field_list.append(val)
        #print('maximum E-field: ', max_E_field, '[kV/m]')
        file.close()
        return(max_E_field,  E_field_list)
        
    except: 
        print("ERROR - couldn't find file!")
        

In [ ]:
# Example of running this function
#E_field_line('TRAK_tools_xopt_test_2/a012d010_20_200_670_250', 4.0, plots = True)

In [ ]:
## Function to get the current density plot (for file defined and at slice defined)
def cdensity(TLS_file, dens_slice):  #example: cdensity('TRAK_auto_10_03/TRAK_demit_007', 775)
    try:
        file = open(r'C://fieldp_pro/tricomp/Automation/sim_automation/%s.TLS' %TLS_file)
        lines = file.readlines()

        dens_sci =  "{:.5E}".format(dens_slice)
        dens_sci_str = str(dens_sci)

        Current_density = []
        i,j = 0,0
        for line in lines: 
            if 'Axial position, X:  %s'%dens_sci_str in line:   #the number to match where the cdens is
                j = 1
            if (j ==1) and ('Line' in line): 
               j = 0
            if j == 1: 
                Current_density.append(line)
            i += 1
            
        Current_density = Current_density[3:]
        
        if Current_density == []:
            print("No current density slice found here")
        else:
            r = []
            Jz = []
            for line in Current_density: 
                separate  = line.split()
                r.append(float(separate[0]))
                Jz.append(float(separate[1]))

            plt.plot(r, Jz, '.')
            plt.xlabel("Radius [mm]")
            plt.ylabel(r"Current Density $[\frac{A}{m^2}]$ ")
            plt.title("Current Density Slice at Z = %s mm" %dens_sci_str)
            plt.savefig('%s_cdens.png' %TLS_file)
        return(r, Jz)
    except: 
        print("ERROR - couldn't find file!")

In [ ]:
# Example of running cdensity function
#cdensity('TRAK_auto_10_03/TRAK_demit_007', 775)

In [ ]:
## function to call cdensity with a set radius and calculate the max difference in cdensity
## (for a measure of flatness!)
def dist_flatness(TLS_file, dens_slice, radius):
    try: 
        density_list = cdensity(TLS_file, dens_slice)
        plt.close()
        r = density_list[0]
        Jz = density_list[1]
        Jz_main = []
        for i, j in zip(r, Jz): 
            if i > radius: 
                break
            Jz_main.append(j)
            
        import statistics 
        avg = np.round(statistics.mean(Jz_main), 4)
        #  get the minimum and maximum density values
        min_Jz = min(Jz_main)
        max_Jz = max(Jz_main)
        Jz_range = np.round(max_Jz - min_Jz, 4)
        return(avg, Jz_range)
    except: 
        print('ERROR!')

In [ ]:
# Example of running dist_flatness function
#dist_flatness('TRAK_auto_10_03/TRAK_demit_007', 775, 12)

In [ ]:
#Function to change the TRAK .TIN demit parameter (with given input and output filenames)
def demit_func(file_in, file_out, demit):
    demit_str = str(demit)
    demit_sci =  "{:E}".format(demit)
    demit_sci_str = str(demit_sci)
    demit_file = demit_str.replace('.', '')

    file_in_tin = '%s.TIN' %file_in
    file_out_tin = '%s.TIN' %file_out

    with open(file_in_tin, 'r') as f_in:
        with open(file_out_tin, 'w') as f_out:
            for line in f_in:
                if '*File' in line: 
                    line_fix = '*File = %s' %file_out_tin
                    f_out.write(line.replace(line, line_fix))

                elif 'EMIT' in line: 
                    line_list = line.split()
                    line_fix = sub(line_list[3], '%s' %demit_sci_str, line) ## Change it so can change line
                    f_out.write(line.replace(line, line_fix))

                elif 'PARTFILE' in line:
                    line_list = line.split()
                    line_fix = sub(line_list[1], '%s' %file_out, line)
                    f_out.write(line.replace(line, line_fix ))

                elif 'EDUMP' in line:
                    line_list = line.split()
                    line_fix = sub(line_list[1], '%s' %file_out, line)
                    f_out.write(line.replace(line, line_fix)) 

                else:
                    line_fix = line
                    f_out.write(line.replace(line, line_fix))

In [ ]:
# Example of running this function
#demit_func('TRAK_auto_10_03/TRAK_demit_004','TRAK_auto_10_03/TRAK_demit_003_what', 0.03 )

In [ ]:
#Function to change the TRAK .TIN AVG parameter (with given input and output filenames)
def avg_func(file_in, file_out, avg):
    avg_str = str(avg)
    avg_sci =  "{:E}".format(avg)
    avg_sci_str = str(avg_sci)
    avg_file = avg_str.replace('.', '')

    file_in_tin = '%s.TIN' %file_in
    file_out_tin = '%s.TIN' %file_out

    with open(file_in_tin, 'r') as f_in:
            with open(file_out_tin, 'w') as f_out:
                for line in f_in:
                    if '*File' in line: 
                        line_fix = '*File = %s'%file_out_tin
                        f_out.write(line.replace(line, line_fix))

                    elif 'AVG' in line: 
                        line_list = line.split()
                        line_fix = sub(line_list[1], '%s' %avg_sci_str, line)
                        f_out.write(line.replace(line, line_fix))

                    elif 'PARTFILE' in line:
                        line_list = line.split()
                        line_fix = sub(line_list[1], '%s' %file_out, line)
                        f_out.write(line.replace(line, line_fix ))

                    elif 'EDUMP' in line:
                        line_list = line.split()
                        line_fix = sub(line_list[1], '%s' %file_out, line)
                        f_out.write(line.replace(line, line_fix)) 

                    else:
                        line_fix = line
                        f_out.write(line.replace(line, line_fix))
    return('%s file created!' %file_out)

In [ ]:
# Example of running this function
#avg_func('TRAK_auto_10_05/TRAK_avg_017', 'TRAK_auto_10_05/TRAK_avg_018_what', 0.18)

In [ ]:
# Function to change the TRAK .TIN avg and demit parameters: 
#Function to change the TRAK .TIN AVG parameter (with given input and output filenames)
def avg_demit_ncycle_func(file_in, file_out, avg, demit, ncycle):
    avg_str = str(avg)
    avg_sci =  "{:E}".format(avg)
    avg_sci_str = str(avg_sci)
    avg_file = avg_str.replace('.', '')
    
    demit_str = str(demit)
    demit_sci =  "{:E}".format(demit)
    demit_sci_str = str(demit_sci)
    demit_file = demit_str.replace('.', '')
    
    ncycle_str = str(ncycle)

    file_in_tin = '%s.TIN' %file_in
    file_out_tin = '%s.TIN' %file_out

    with open(file_in_tin, 'r') as f_in:
            with open(file_out_tin, 'w') as f_out:
                for line in f_in:
                    if '*File' in line: 
                        line_fix = '*File = %s \n'%file_out_tin
                        f_out.write(line.replace(line, line_fix))

                    elif 'AVG' in line: 
                        line_list = line.split()
                        line_fix = sub(line_list[1], '%s' %avg_sci_str, line)
                        f_out.write(line.replace(line, line_fix))
                        
                    elif 'EMIT' in line: 
                        line_list = line.split()
                        line_fix = sub(line_list[3], '%s' %demit_sci_str, line) ## Change it so can change line
                        f_out.write(line.replace(line, line_fix))
                        
                    elif 'NCYCLE' in line: 
                        line_list = line.split()
                        line_fix = sub(line_list[1], '%s' %ncycle_str, line)
                        f_out.write(line.replace(line, line_fix))

                    elif 'PARTFILE' in line:
                        line_list = line.split()
                        line_fix = sub(line_list[1], '%s' %file_out, line)
                        f_out.write(line.replace(line, line_fix ))

                    elif 'EDUMP' in line:
                        line_list = line.split()
                        line_fix = sub(line_list[1], '%s' %file_out, line)
                        f_out.write(line.replace(line, line_fix)) 

                    else:
                        line_fix = line
                        f_out.write(line.replace(line, line_fix))
    return('%s file created!' %file_out)

In [ ]:
# Example of running this function
#avg_demit_ncycle_func('TRAK_auto_10_06/TRAK_avg004_demit_05_ncycle70','TRAK_auto_10_06/TRAK_avg005_demit01_ncycle45',0.05, 0.1, 45)

In [ ]:
## Function to check if the current converges for each sim (IN PROGRESS!!) 
# input list of files you want to check on current convergence and retreive final current 
def converg_func(file_list, conv_val):
    try:
        tls_file_conv_list = []
        for file_name in file_list:
            file = open(r'C://fieldp_pro/tricomp/Automation/sim_automation/%s.TLS' %file_name) 
            lines = file.readlines()
            CONVERGE = True # Set the convergence status to be true as default

            Current_convergence = []
            i = 0
            j = 0
            for line in lines: 
                if 'Current convergence history' in line: 
                    j = 1
                if 'Ending time' in line: 
                    j = 0
                if j == 1: 
                    Current_convergence.append(lines[i])
                i += 1
                
            Current_convergence = Current_convergence[4:-1]

            NCycle = []
            Total_emitted_current = []
            for line in Current_convergence: 
                separate  = line.split()
                NCycle.append(float(separate[0]))
                Total_emitted_current.append(float(separate[1]))

            # Check for any outliers in the current convergence! 
            for i in range(-5, -1, 1):
                current = Total_emitted_current[i] 
                current_after = Total_emitted_current[i + 1]
                current_before = Total_emitted_current[i - 1]
                current_diff_after = abs(current - current_after)
                current_diff_before = abs(current - current_before)
                if current_diff_after > conv_val or current_diff_before > conv_val: 
                    CONVERGE = False
                    
            tls_file_conv_list.append([file_name, CONVERGE, Total_emitted_current[-1]])
    
            tls_file_conv_list_true = []
            for i in range(len(tls_file_conv_list)): 
                if tls_file_conv_list[i][1] == True: 
                    tls_file_conv_list_true.append(tls_file_conv_list[i])
                    
        print(tls_file_conv_list_true)
        return(tls_file_conv_list, tls_file_conv_list_true)
    
    except: 
        print('ERRROR - issue with finding file or performing operation')

## NOW FOR THE NEW GEOMETRY MESH FILE MAKING

In [ ]:
#EXPANDING THE CATHODE SURFACE!! - ALSO EXTEND THE FOCUS AND SUCH 
def mesh_file_electrode_spacing2(fe_x, fe_y, an_x, an_y, file_to_save):
    
    # focus eletrode dimensions 
    fe_x_sci =  "{:.4E}".format(fe_x)
    fex = str(fe_x_sci)
    #print(fex)
    
    fe_x2 = fe_x - 1.4
    fex2_sci = "{:.4E}".format(fe_x2)
    fex2 = str(fex2_sci)
    
    fe_x3 = fe_x - 0.5
    fex3_sci = "{:.4E}".format(fe_x3)
    fex3 = str(fex3_sci)
    
    fe_x4 = fe_x - 5.1
    fex4_sci = "{:.4E}".format(fe_x4)
    fex4 = str(fex4_sci)
    
    fe_y_sci = "{:.4E}".format(fe_y)
    fey = str(fe_y_sci)
    #print(fey)
    
    fe_y2 = fe_y + 0.89
    fe_y2_sci = "{:.4E}".format(fe_y2)
    fey2 = str(fe_y2_sci)
    
    fe_y3 = fe_y + 0.5
    fe_y3_sci = "{:.4E}".format(fe_y3)
    fey3 = str(fe_y3_sci)
    
    fe_y4 = fe_y + 1.9
    fe_y4_sci = "{:.4E}".format(fe_y4)
    fey4 = str(fe_y4_sci)
    
    # anode dimensions! 
    an_x_sci = "{:.4E}".format(an_x)
    anx = str(an_x_sci)
    #print(anx)
    
    an_x2 = an_x - 3.5
    an_x2_sci = "{:.4E}".format(an_x2)
    anx2 = str(an_x2_sci)
    
    an_x3 = an_x - 29.98
    an_x3_sci = "{:.4E}".format(an_x3)
    anx3 = str(an_x3_sci)
    
    an_x4 = an_x + 2.6
    an_x4_sci = "{:.4E}".format(an_x4)
    anx4 = str(an_x4_sci)
    
    an_x5 = an_x + 65.45
    an_x5_sci = "{:.4E}".format(an_x5)
    anx5 = str(an_x5_sci)
    
    an_x6 = an_x + 8.49
    an_x6_sci = "{:.4E}".format(an_x6)
    anx6 = str(an_x6_sci)
    
    an_x7 = an_x + 3.37
    an_x7_sci = "{:.4E}".format(an_x7)
    anx7 = str(an_x7_sci)
    
    an_x8 = an_x - 8.49
    an_x8_sci = "{:.4E}".format(an_x8)
    anx8 = str(an_x8_sci)
    
    an_x9 = an_x - 16.98
    an_x9_sci = "{:.4E}".format(an_x9)
    anx9 = str(an_x9_sci)
    
    an_x10 = an_x - 18.48
    an_x10_sci = "{:.4E}".format(an_x10)
    anx10 = str(an_x10_sci)
    
    an_x11 = an_x - 14.98
    an_x11_sci = "{:.4E}".format(an_x11)
    anx11 = str(an_x11_sci)
    
    an_x12 = an_x - 4.5
    an_x12_sci = "{:.4E}".format(an_x12)
    anx12 = str(an_x12_sci)
    
    
    #anode y positions! 
    an_y_sci = "{:.4E}".format(an_y)
    anyy = str(an_y_sci)
    #print(anyy)
    
    an_y2 = an_y + 6.96 + 10 - 1.04
    an_y2_sci = "{:.4E}".format(an_y2)
    any2 = str(an_y2_sci)
    
    an_y3 = an_y + 8 + 10
    an_y3_sci = "{:.4E}".format(an_y3)
    any3 = str(an_y3_sci)
    
    an_y4 = an_y + 14.72
    an_y4_sci = "{:.4E}".format(an_y4)
    any4 = str(an_y4_sci)
    
    an_y5 = an_y + 9.5 + 10
    an_y5_sci = "{:.4E}".format(an_y5)
    any5 = str(an_y5_sci)
    
    an_y6 = an_y + 15.99
    an_y6_sci = "{:.4E}".format(an_y6)
    any6 = str(an_y6_sci)
    
    an_y7 = an_y + 2.85
    an_y7_sci = "{:.4E}".format(an_y7)
    any7 = str(an_y7_sci)
    
    an_y8 = an_y + 1.5
    an_y8_sci = "{:.4E}".format(an_y8)
    any8 = str(an_y8_sci)
    
    an_y9 = an_y - 0.5
    an_y9_sci = "{:.4E}".format(an_y9)
    any9 = str(an_y9_sci)
    
    an_y10 = an_y - 5.62
    an_y10_sci = "{:.4E}".format(an_y10)
    any10 = str(an_y10_sci)
    
    an_y11 = an_y - 6.23
    an_y11_sci = "{:.4E}".format(an_y11)
    any11 = str(an_y11_sci)
    
    an_y12 = an_y - 4.73
    an_y12_sci = "{:.4E}".format(an_y12)
    any12 = str(an_y12_sci)
    
    an_y13 = an_y - 4.4
    an_y13_sci = "{:.4E}".format(an_y13)
    any13 = str(an_y13_sci)
    
    an_y14 = an_y - 0.9
    an_y14_sci = "{:.4E}".format(an_y14)
    any14 = str(an_y14_sci)
    
    
    with open(r'C://fieldp_pro/tricomp/Automation/sim_automation/%s.MIN' %file_to_save, 'w') as f:
         f.write('* File: %s.MIN \n'  %file_to_save)
         f.write('* MESH script (Field Precision) \n')
         f.write('''
        
         *  -------------------------------------------------------
GLOBAL
  ZMESH
      -28.00000  40.00000    0.1000
	   40.000000  290.0000    1.0000
  END
  RMESH
        0.00000  60.00000    0.1000
        60.0000  250.0000    1.0000
  END
END
*  -------------------------------------------------------
REGION  FILL VACUUM
  L   -2.800E+01  2.500E+02 -2.800E+01  0.000E+00
  L   -2.800E+01  0.000E+00  290	    0.000E+00
  L    290        0.000E+00  290        2.500E+02
  L    290        2.500E+02 -2.800E+01  2.500E+02
END 
*  -------------------------------------------------------
REGION  FILL CHAMBERWALL
  L   -2.800E+01  6.000E+01  290        6.000E+01
  L    290        6.000E+01  290        6.200E+01
  L    290        6.200E+01 -2.800E+01  6.200E+01
  L   -2.800E+01  6.200E+01 -2.800E+01  6.000E+01
END 
*  -------------------------------------------------------
REGION  FILL CATHODE
  L   -2.800E+01  0.0000000 0.00000000  0.0000000
  L   0.00000000  0.0000000 0.00000000    1.000E+01
  A   0.00000000  1.000E+01 -20.000E-01  1.200E+01 -20.000E-01  1.000E+01
  L   -20.000E-01  1.200E+01 -2.800E+01  1.200E+01
  L   -2.800E+01  1.200E+01 -2.800E+01  0.0000000
END 
*  -------------------------------------------------------
REGION EMISSION
  L	0.00000000  0.0000000 0.00000000  1.000E+01
  A   0.00000000  1.000E+01 -15.000E-01  1.1936E+01 -20.000E-01  1.000E+01

END 
*  -------------------------------------------------------
REGION  FILL FOCUS
  L   -2.800E+01  %s %s %s
  L   %s  %s %s %s
  L   %s  %s  %s  %s
  A    %s  %s  %s  %s  %s %s
  A    %s  %s %s  %s  %s  %s
  L   %s  %s -2.800E+01  %s
  L   -2.800E+01  %s -2.800E+01  %s
END 
*  -------------------------------------------------------
         ''' %(fey2, fex4, fey2,
                 fex4, fey2, fex4, fey,
                 fex4, fey, fex3, fey,
                 fex3, fey, fex, fey3, fex3, fey3,
                 fex, fey3, fex2, fey4, fex2, fey3,
                 fex2,fey4, fey4,
                 fey4, fey2))
                 
         f.write('''
REGION  FILL SHIELD
  L   -2.800E+01  3.999E+01 -2.520E+00  3.999E+01
  L   -2.520E+00  3.999E+01 -2.520E+00  4.249E+01
  L   -2.520E+00  4.249E+01 -2.800E+01  4.249E+01
  L   -2.800E+01  4.249E+01 -2.800E+01  3.999E+01
END 
*  -------------------------------------------------------
REGION  FILL CONTROL
  L   -2.800E+01  4.350E+01  1.220E+00  4.350E+01
  A    1.220E+00  4.350E+01  1.220E+00  4.500E+01  1.220E+00  4.425E+01
  L    1.220E+00  4.500E+01 -2.800E+01  4.500E+01
  L   -2.800E+01  4.500E+01 -2.800E+01  4.350E+01
END 
*  -------------------------------------------------------
         ''')
         f.write('''
REGION  FILL ANODESUPPORT
  L   -2.800E+01  %s -1.958E+01  %s
  L   -1.958E+01  %s %s  %s
  L   %s  %s %s  %s
  L   %s  %s  %s  %s
  L    %s  %s  %s  %s
  L    %s  %s  %s  %s
  L    %s  %s  %s  %s
  L    %s  %s  %s  %s
  L   %s  %s -2.800E+01  %s
  L   -2.800E+01  %s -2.800E+01  %s
END 
*  -------------------------------------------------------
         ''' %(any3, any3,
             any3, anx3, any3,
             anx3, any3, anx3, any2,
             anx3, any2, anx2, any2,
             anx2, any2, anx2, anyy,
             anx2, anyy, anx, anyy,
             anx, anyy, anx, any5,
             anx, any5, anx3, any5,
             anx3, any5, any5,
             any5, any3,
         ))
         f.write('''
REGION  FILL ANODE
  L    %s  %s  %s  %s
  L    %s  %s  %s  %s
  L    %s  %s  %s  %s
  L    %s  %s  %s  %s
  L    %s  %s  %s  %s
  L    %s  %s  %s %s
  L    %s  %s  %s  %s
  L    %s  %s  %s  %s
  L    %s  %s  %s  %s
  L    %s  %s  %s  %s
  L    %s  %s  %s  %s
  L    %s  %s  %s  %s
  A    %s  %s  %s  %s  %s  %s
  L    %s %s  %s  %s
  A    %s  %s  %s  %s  %s  %s
END 
*  -------------------------------------------------------
         ''' %(anx9, any11, anx8, any11,
             anx8, any11, anx7, any10,
             anx7, any10, anx6, any9,
             anx6, any9, anx5, any9,
             anx5, any9, anx5, any8, 
             anx5, any8, anx4, any8,
             anx4, any8, anx4, any7,
             anx4, any7, anx, any7,
             anx, any7, anx, anyy, 
             anx, anyy, anx12, anyy,
             anx12, anyy, anx12, any14,
             anx12, any14, anx11, any14,
             anx11, any14, anx10, any13, anx11, any13, 
             anx10, any13, anx10, any12,
             anx10, any12, anx9, any11, anx9, any12
         ))
         f.write('''
ENDFILE
         
         ''')

In [ ]:
# Example of running this function
#mesh_file_electrode_spacing2(6, 3, 51, 6, 'new_electrode_spacing')

## NOW FOR THE NEW GEOMETRY ESTAT FILE MAKING

In [ ]:
# write estat file for the mesh file! 
# FOR WITH ANODE BUT WITHOUT GUN EXIT TUBE 
def estat_file2(mesh_file, estat_file, emission_potential, fe_potential):  #input the mesh file to run, estat file to save, and emission potential!

    with open(r'C://fieldp_pro/tricomp/Automation/sim_automation/'+estat_file+'.EIN', 'w') as f:
        f.write('* File: %s.EIN \n'  %estat_file)
        f.write('* ESTAT script (Field Precision) \n')
        f.write('Mesh = %s \n'%mesh_file)
        f.write('''
    Geometry = Cylin
    DUnit =   1.0000E+03
    ResTarget =   5.0000E-08
    MaxCycle =    5000

    * Region 1: VACUUM
    Epsi(1) =   1.0000E+00

    * Region 2: CHAMBERWALL
    Potential(2) =   0.0000E+00

    * Region 3: CATHODE
    Potential(3) =  %s

    * Region 4: EMISSION
    Potential(4) =  %s

    * Region 5: FOCUS
    Potential(5) =  %s

    * Region 6: SHIELD
    Potential(6) =  %s

    * Region 7: CONTROL
    Potential(7) =  %s

    * Region 8: ANODESUPPORT
    Potential(8) =   0.0000E+00

    * Region 9: ANODE
    Potential(9) =   0.0000E+00


    EndFile

        ''' %(emission_potential, emission_potential, fe_potential, fe_potential, fe_potential))

In [ ]:
## Example of running this fucnction! 
#estat_file2('TRAK_auto_10_05/electrode_spacing_2', 'estat_file_10_19', '-1.36E+03')

## NOW FOR THE NEW GEOMETRY PERMAG MESH FILE MAKING

In [1]:
# write the mesh file for permag! 
# Write a mesh for the permag solution

def mesh_permag_file2(mesh_permag_filename):
    with open(r'C://fieldp_pro/tricomp/Automation/sim_automation/'+mesh_permag_filename+'.MIN', 'w') as f:
        f.write('* File: %s.MIN \n'  %mesh_permag_filename)
        f.write('* MESH script (Field Precision) \n')
        f.write('''
        *  -------------------------------------------------------
GLOBAL
  ZMESH
     -21.000E+01  2.90E+02  2.000E+00
  END
  RMESH
      0.000E+00  2.500E+02  2.000E+00
  END
END
*  -------------------------------------------------------
REGION FILL VACUUM
  L   -21.000E+01  2.500E+02 -21.000E+01  0.000E+00
  L   -21.000E+01  0.000E+00  2.900E+02  0.000E+00
  L    2.900E+02   0.000E+00  2.900E+02  2.500E+02
  L    2.900E+02   2.500E+02 -21.000E+01  2.500E+02
END
*  -------------------------------------------------------
REGION FILL SOLENOID1
  L	-210		195		290		195
  L 	290		195		290		250
  L	290		250		-210		250
  L	-210		250		-210		195
END 
*  -------------------------------------------------------
ENDFILE
        ''')

In [ ]:
# Example for running this function! 
#mesh_permag_file2('TRAK_auto_10_25/mesh_permag')

## NOW FOR THE NEW GEOMETRY PERMAG FILE MAKING

In [ ]:
#write the permag file for the solenoid
def permag_file2(meshpermag_filename, permag_filename):
    with open(r'C://fieldp_pro/tricomp/Automation/sim_automation/'+permag_filename+'.PIN', 'w') as f:
        f.write('* File: %s.PIN \n'  %permag_filename)
        f.write('* PerMag script (Field Precision) \n')
        f.write('Mesh =  %s \n'%meshpermag_filename)
        f.write('''
Geometry = Cylin
DUnit =   1.0000E+03
ResTarget =   1.0000E-09
MaxCycle =    5000
Avg =   1.0000E-01
Update =     10   200

* Region 1: VACUUM
Mu(1) =   1.0000E+00

* Region 2: SOLENOID1
Current(2) =   119366.21

EndFile
        ''')

In [ ]:
# Example for running this function! 
#permag_file2('TRAK_auto_10_25/meshpermag', 'TRAK_auto_10_25/permag')

## NOW FOR THE NEW GEOMETRY TRAK FILE MAKING

In [ ]:
# write TRAK file with the given estat and permag files 

def TRAK_file2(estat_filename, permag_filename, TRAK_filename):
    with open(r'C://fieldp_pro/tricomp/Automation/sim_automation/'+TRAK_filename+'.TIN', 'w') as f: 
        f.write('* Trak script (Field Precision) \n')
        f.write("*File: %s.TIN \n" %TRAK_filename)
        f.write('''
FIELDS
  EFILE: %s.eou
  BFILE: %s.pou
  DUNIT:   1.0000E+03
  MAXCYCLE:    350
  RESTARGET:   2.0000E-07
END

PARTICLES RELBEAM
  NCYCLE:   15
  AVG:   1.5000E-01
  DT: 4.0000E-12
  EMIT(4):   0.0000E+00 -1.0000E+00  0.1500     6
END

DIAGNOSTICS
  PARTFILE: %s
  EDUMP: %s
  
  ESCAN (-25, 0, 0)(-25, 60, 0)
  ESCAN (-20, 0, 0)(-20, 60, 0)
  ESCAN (-15, 0, 0)(-15, 60, 0)
  ESCAN (-10, 0, 0)(-10, 60, 0)
  ESCAN (-5, 0, 0)(-5, 60, 0)
  ESCAN (0, 0, 0)(0, 60, 0)
  ESCAN (5, 0, 0)(5, 60, 0)
  ESCAN (10, 0, 0)(10, 60, 0)
  ESCAN (15, 0, 0)(15, 60, 0)
  ESCAN (20, 0, 0)(20, 60, 0)
  ESCAN (25, 0, 0)(25, 60, 0)
  ESCAN (30, 0, 0)(30, 60, 0)
  ESCAN (31, 0, 0)(31, 60, 0)
  ESCAN (32, 0, 0)(32, 60, 0)
  ESCAN (33, 0, 0)(33, 60, 0)
  ESCAN (34, 0, 0)(34, 60, 0)
  ESCAN (35, 0, 0)(35, 60, 0)
  ESCAN (40, 0, 0)(40, 60, 0)
  ESCAN (45, 0, 0)(45, 60, 0)
  ESCAN (50, 0, 0)(50, 60, 0)
  ESCAN (55, 0, 0)(55, 60, 0)
  ESCAN (60, 0, 0)(60, 60, 0)
  ESCAN (65, 0, 0)(65, 60, 0)
  ESCAN (70, 0, 0)(70, 60, 0)
  ESCAN (75, 0, 0)(75, 60, 0)
  
  
  PARTLIST Cylin
  CDENS  150 20 80
  CDENS  50  20 80
  CDENS  250 20 80
END

ENDFILE
        ''' %(estat_filename, permag_filename, TRAK_filename, TRAK_filename))

In [ ]:
# Example for running this function! 
#TRAK_file2('TRAK_auto_10_25/estat_5_2_50_5', 'TRAK_auto_10_25/permag', 'TRAK_auto_10_25/TRAKFILE')

In [ ]:
## call a TRAK command to run the simulations in parallel
## give the list of files to run (with directory precursor), the number of sims to run in parallel, and the trak command to run

def run_parallel(file_list, parallel_runs, command):
    import subprocess 
    string_list = ','.join(str(elem) for elem in file_list)
    result = subprocess.run(['python', 'run_trak_cmd.py', '%s' %string_list, '%s' %parallel_runs, '%s' %command], stdout = subprocess.PIPE)
    print(result.stdout.decode('utf-8'))

In [ ]:
# Example for running this function! 
#run_parallel(['TRAK_auto_11_2/TRAK_5_2_50_5_demit_009', 'TRAK_auto_11_2/TRAK_5_2_50_5_demit_01', 
#         'TRAK_auto_11_2/TRAK_5_2_50_5_demit_011', 'TRAK_auto_11_2/TRAK_5_2_50_5_demit_012'], '2', 'TRAK')